In [2]:
import torch

In [35]:
data = 'I love neural networks.'
vocabulary = set()
for char in list(data):
    vocabulary.add(char)
vocabulary = list(vocabulary)
input_dim = len(vocabulary) + 1

In [61]:
class Data(torch.utils.data.Dataset):
    def __init__(self, data, vocabulary, input_dim):
        super().__init__()
        self.X, self.y = [], []
        self.input_dim = input_dim
        self.vocabulary = vocabulary
        for i in range(len(data) - 1):
            self.X.append(self.onehotencoding(data[i]))
            self.y.append(self.vocabulary.index(data[i + 1]))
        self.X.append(self.onehotencoding(data[-1]))
        self.y.append(len(vocabulary))
        self.y = torch.tensor(self.y)

    def onehotencoding(self, char):
        ret = torch.zeros(self.input_dim)
        if char == '.':
            ret[-1] = 1
        if char in vocabulary:
            ret[self.vocabulary.index(char)] = 1
        else:
            ret[-2] = 1
        return ret
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [62]:
alldata = Data(data= data, vocabulary= vocabulary, input_dim= input_dim)
train, test = torch.utils.data.random_split(alldata, [0.8, 0.2])
train_loader, test_loader = torch.utils.data.DataLoader(train, batch_size= 2, shuffle= True), torch.utils.data.DataLoader(test, batch_size= 2)

In [63]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size= input_size, hidden_size= 10, num_layers= 1, batch_first= True)
        self.fc1 = torch.nn.Linear(in_features= 10, out_features= output_size, bias= True)
    
    def forward(self, x):
        y, h = self.lstm(x)
        y = y[: , -1, :]
        return self.fc1(torch.sigmoid(y))

In [64]:
model = LSTMModel(input_size= input_dim, output_size= len(vocabulary) + 1)
model = model.to('cuda')

In [65]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [66]:
model.train()
for epoch in range(150):
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.reshape((-1, 1, input_dim)).to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    print(f'epoch = {epoch}, loss = {running_loss}')

epoch = 0, loss = 30.22661519050598
epoch = 1, loss = 29.82709527015686
epoch = 2, loss = 30.017263174057007
epoch = 3, loss = 29.170387744903564
epoch = 4, loss = 28.960737228393555
epoch = 5, loss = 28.94461154937744
epoch = 6, loss = 28.64020347595215
epoch = 7, loss = 28.737663745880127
epoch = 8, loss = 28.66983962059021
epoch = 9, loss = 28.329877614974976
epoch = 10, loss = 28.56909489631653
epoch = 11, loss = 28.405799865722656
epoch = 12, loss = 28.25827717781067
epoch = 13, loss = 27.644880533218384
epoch = 14, loss = 27.850624799728394
epoch = 15, loss = 27.617063522338867
epoch = 16, loss = 27.466607093811035
epoch = 17, loss = 27.668621063232422
epoch = 18, loss = 27.398127794265747
epoch = 19, loss = 27.10334801673889
epoch = 20, loss = 27.103357553482056
epoch = 21, loss = 26.993833303451538
epoch = 22, loss = 26.988488912582397
epoch = 23, loss = 26.759125232696533
epoch = 24, loss = 26.67352294921875
epoch = 25, loss = 26.640578746795654
epoch = 26, loss = 26.909163713

In [69]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.reshape((-1, 1, input_dim)).to('cuda'), target.to('cuda')
        output = model(input)
        val, ind = torch.max(output, dim= 1)
        all_preds.extend(ind.to('cpu'))
        all_labels.extend(target.to('cpu'))
from sklearn.metrics import accuracy_score
print(accuracy_score(all_labels, all_preds))


0.25
